In [3]:
#%loadpy ../Coords/Sun2.py

In [9]:
#!/usr/bin/env python 

""" 
SUNRISET.C - computes Sun rise/set times, start/end of twilight, and
             the length of the day at any date and latitude
			 
Written as DAYLEN.C, 1989-08-16

Modified to SUNRISET.C, 1992-12-01
			 
(c) Paul Schlyter, 1989, 1992
			 
Released to the public domain by Paul Schlyter, December 1992
			 
Direct conversion to Java 
Sean Russell <ser@germane-software.com>
"""
#Conversion to Python Class, 2002-03-21
#Henrik Ha"rko"nen <radix@kortis.to>


import math

class Sun:
    
    def __init__(self):
	""""""
	
	# Some conversion factors between radians and degrees
	self.PI = 3.1415926535897932384
	self.RADEG = 180.0 / self.PI
	self.DEGRAD = self.PI / 180.0
	self.INV360 = 1.0 / 360.0
	

    def daysSince2000Jan0(self, y, m, d):
	"""A macro to compute the number of days elapsed since 2000 Jan 0.0
	   (which is equal to 1999 Dec 31, 0h UT)"""
	return (367*(y)-((7*((y)+(((m)+9)/12)))/4)+((275*(m))/9)+(d)-730530)
    

    # The trigonometric functions in degrees
    def sind(self, x):
	"""Returns the sin in degrees"""
	return math.sin(x * self.DEGRAD)

    def cosd(self, x):
	"""Returns the cos in degrees"""
	return math.cos(x * self.DEGRAD)

    def tand(self, x):
	"""Returns the tan in degrees"""
	return math.tan(x * self.DEGRAD)

    def atand(self, x):
	"""Returns the arc tan in degrees"""
	return math.atan(x) * self.RADEG
    
    def asind(self, x):
	"""Returns the arc sin in degrees"""
	return math.asin(x) * self.RADEG

    def acosd(self, x):
	"""Returns the arc cos in degrees"""
	return math.acos(x) * self.RADEG

    def atan2d(self, y, x):
	"""Returns the atan2 in degrees"""
	return math.atan2(y, x) * self.RADEG

    # Following are some macros around the "workhorse" function __daylen__ 
    # They mainly fill in the desired values for the reference altitude    
    # below the horizon, and also selects whether this altitude should     
    # refer to the Sun's center or its upper limb.                         

    def dayLength(self, year, month, day, lon, lat):
	"""
	This macro computes the length of the day, from sunrise to sunset.
	Sunrise/set is considered to occur when the Sun's upper limb is
	35 arc minutes below the horizon (this accounts for the refraction
	of the Earth's atmosphere).
	"""
	return self.__daylen__(year, month, day, lon, lat, -35.0/60.0, 1)
    
    def dayCivilTwilightLength(self, year, month, day, lon, lat):
	"""
	This macro computes the length of the day, including civil twilight.
	Civil twilight starts/ends when the Sun's center is 6 degrees below
	the horizon. 
	"""
	return self.__daylen__(year, month, day, lon, lat, -6.0, 0)

    def dayNauticalTwilightLength(self, year, month, day, lon, lat):
	"""
	This macro computes the length of the day, incl. nautical twilight.
	Nautical twilight starts/ends when the Sun's center is 12 degrees
	below the horizon.
	"""
	return self.__daylen__(year, month, day, lon, lat, -12.0, 0)

    def dayAstronomicalTwilightLength(self, year, month, day, lon, lat):
	"""
	This macro computes the length of the day, incl. astronomical twilight.
	Astronomical twilight starts/ends when the Sun's center is 18 degrees 
	below the horizon. 
	"""
	return self.__daylen__(year, month, day, lon, lat, -18.0, 0)
    
    def sunRiseSet(self, year, month, day, lon, lat):
	"""
	This macro computes times for sunrise/sunset.
	Sunrise/set is considered to occur when the Sun's upper limb is
	35 arc minutes below the horizon (this accounts for the refraction
	of the Earth's atmosphere).
	"""
	return self.__sunriset__(year, month, day, lon, lat, -35.0/60.0, 1)

    
    def civilTwilight(self, year, month, day, lon, lat):
	"""
	This macro computes the start and end times of civil twilight. 
	Civil twilight starts/ends when the Sun's center is 6 degrees below 
	the horizon.
	"""
	return self.__sunriset__(year, month, day, lon, lat, -6.0, 0)
    
    def nauticalTwilight(self, year, month, day, lon, lat):
	"""
	This macro computes the start and end times of nautical twilight.
	Nautical twilight starts/ends when the Sun's center is 12 degrees
	below the horizon.
	"""
	return self.__sunriset__(year, month, day, lon, lat, -12.0, 0)
    
    def astronomicalTwilight(self, year, month, day, lon, lat):
	"""
	This macro computes the start and end times of astronomical twilight.
	Astronomical twilight starts/ends when the Sun's center is 18 degrees
	below the horizon.
	"""
	return self.__sunriset__(year, month, day, lon, lat, -18.0, 0)
    
    # The "workhorse" function for sun rise/set times
    def __sunriset__(self, year, month, day, lon, lat, altit, upper_limb):
	"""
	Note: year,month,date = calendar date, 1801-2099 only.
	      Eastern longitude positive, Western longitude negative
              Northern latitude positive, Southern latitude negative
	      The longitude value IS critical in this function!
	      altit = the altitude which the Sun should cross
	              Set to -35/60 degrees for rise/set, -6 degrees
		      for civil, -12 degrees for nautical and -18
		      degrees for astronomical twilight.
	        upper_limb: non-zero -> upper limb, zero -> center
		      Set to non-zero (e.g. 1) when computing rise/set
		      times, and to zero when computing start/end of
		      twilight.
	      *rise = where to store the rise time 
	      *set  = where to store the set  time 
	              Both times are relative to the specified altitude,
		      and thus this function can be used to compute
		      various twilight times, as well as rise/set times
	Return value:  0 = sun rises/sets this day, times stored at
                           *trise and *tset.
		      +1 = sun above the specified 'horizon' 24 hours.
		           *trise set to time when the sun is at south,
			   minus 12 hours while *tset is set to the south
			   time plus 12 hours. 'Day' length = 24 hours 
		      -1 = sun is below the specified 'horizon' 24 hours
		           'Day' length = 0 hours, *trise and *tset are
			    both set to the time when the sun is at south.
	"""
	# Compute d of 12h local mean solar time
	d = self.daysSince2000Jan0(year,month,day) + 0.5 - (lon/360.0)
	
	# Compute local sidereal time of this moment 
	sidtime = self.revolution(self.GMST0(d) + 180.0 + lon)
	
	# Compute Sun's RA + Decl at this moment 
	res = self.sunRADec(d)
	sRA = res[0]
	sdec = res[1]
	sr = res[2]
	
	# Compute time when Sun is at south - in hours UT 
	tsouth = 12.0 - self.rev180(sidtime - sRA)/15.0;
	
	# Compute the Sun's apparent radius, degrees 
	sradius = 0.2666 / sr;
	
	# Do correction to upper limb, if necessary 
	if upper_limb:
	    altit = altit - sradius
	
	# Compute the diurnal arc that the Sun traverses to reach 
	# the specified altitude altit: 
	
	cost = (self.sind(altit) - self.sind(lat) * self.sind(sdec))/(self.cosd(lat) * self.cosd(sdec))

	if cost >= 1.0:
	    rc = -1 
	    t = 0.0           # Sun always below altit
	    
	elif cost <= -1.0:
	    rc = +1 
	    t = 12.0;         # Sun always above altit

	else:
	    t = self.acosd(cost)/15.0   # The diurnal arc, hours

	
	# Store rise and set times - in hours UT 
	return (tsouth-t, tsouth+t)


    def __daylen__(self, year, month, day, lon, lat, altit, upper_limb):
	"""
	Note: year,month,date = calendar date, 1801-2099 only.             
	      Eastern longitude positive, Western longitude negative       
	      Northern latitude positive, Southern latitude negative       
	      The longitude value is not critical. Set it to the correct   
	      longitude if you're picky, otherwise set to to, say, 0.0     
	      The latitude however IS critical - be sure to get it correct 
	      altit = the altitude which the Sun should cross              
	              Set to -35/60 degrees for rise/set, -6 degrees       
	              for civil, -12 degrees for nautical and -18          
	              degrees for astronomical twilight.                   
	        upper_limb: non-zero -> upper limb, zero -> center         
	              Set to non-zero (e.g. 1) when computing day length   
	              and to zero when computing day+twilight length.      
							
	"""
	
	# Compute d of 12h local mean solar time 
	d = self.daysSince2000Jan0(year,month,day) + 0.5 - (lon/360.0)
		 
	# Compute obliquity of ecliptic (inclination of Earth's axis) 
	obl_ecl = 23.4393 - 3.563E-7 * d
	
	# Compute Sun's position 
	res = self.sunpos(d)
	slon = res[0]
	sr = res[1]
	
	# Compute sine and cosine of Sun's declination 
	sin_sdecl = self.sind(obl_ecl) * self.sind(slon)
	cos_sdecl = math.sqrt(1.0 - sin_sdecl * sin_sdecl)
	
	# Compute the Sun's apparent radius, degrees 
	sradius = 0.2666 / sr
	
	# Do correction to upper limb, if necessary 
	if upper_limb:
	    altit = altit - sradius

	    
	cost = (self.sind(altit) - self.sind(lat) * sin_sdecl) / (self.cosd(lat) * cos_sdecl)
	if cost >= 1.0:
	    t = 0.0             # Sun always below altit
	
	elif cost <= -1.0:
	    t = 24.0      # Sun always above altit
	
	else:
	    t = (2.0/15.0) * self.acosd(cost);     # The diurnal arc, hours
	    
	return t

    
    def sunpos(self, d):
	"""
	Computes the Sun's ecliptic longitude and distance 
	at an instant given in d, number of days since     
	2000 Jan 0.0.  The Sun's ecliptic latitude is not  
	computed, since it's always very near 0.           
	"""

	# Compute mean elements 
	M = self.revolution(356.0470 + 0.9856002585 * d)
	w = 282.9404 + 4.70935E-5 * d
	e = 0.016709 - 1.151E-9 * d
	
	# Compute true longitude and radius vector 
	E = M + e * self.RADEG * self.sind(M) * (1.0 + e * self.cosd(M))
	x = self.cosd(E) - e
	y = math.sqrt(1.0 - e*e) * self.sind(E)
	r = math.sqrt(x*x + y*y)              #Solar distance 
	v = self.atan2d(y, x)                 # True anomaly 
	lon = v + w                        # True solar longitude 
	if lon >= 360.0:
	    lon = lon - 360.0   # Make it 0..360 degrees
	    
	return (lon,r)
    

    def sunRADec(self, d):
	""""""
	
	# Compute Sun's ecliptical coordinates 
	res = self.sunpos(d)
	lon = res[0]
	r = res[1]
	
	#Compute ecliptic rectangular coordinates (z=0) 
	x = r * self.cosd(lon)
	y = r * self.sind(lon)
	
	# Compute obliquity of ecliptic (inclination of Earth's axis) 
	obl_ecl = 23.4393 - 3.563E-7 * d
	
	# Convert to equatorial rectangular coordinates - x is unchanged 
	z = y * self.sind(obl_ecl)
	y = y * self.cosd(obl_ecl)

	# Convert to spherical coordinates 
	RA = self.atan2d(y, x)
	dec = self.atan2d(z, math.sqrt(x*x + y*y))

	return (RA, dec, r)


    def revolution(self, x):
	"""
	This function reduces any angle to within the first revolution 
	by subtracting or adding even multiples of 360.0 until the     
	result is >= 0.0 and < 360.0
	
	Reduce angle to within 0..360 degrees
	"""
	return (x - 360.0 * math.floor(x * self.INV360))
    
    def rev180(self, x):
	"""
	Reduce angle to within +180..+180 degrees
	""" 
	return (x - 360.0 * math.floor(x * self.INV360 + 0.5))

    def GMST0(self, d):
	"""
	This function computes GMST0, the Greenwich Mean Sidereal Time  
	at 0h UT (i.e. the sidereal time at the Greenwhich meridian at  
	0h UT).  GMST is then the sidereal time at Greenwich at any     
	time of the day.  I've generalized GMST0 as well, and define it 
	as:  GMST0 = GMST - UT  --  this allows GMST0 to be computed at 
	other times than 0h UT as well.  While this sounds somewhat     
	contradictory, it is very practical:  instead of computing      
	GMST like:                                                      
	                                                                
	 GMST = (GMST0) + UT * (366.2422/365.2422)                      
	                                                                
	where (GMST0) is the GMST last time UT was 0 hours, one simply  
	computes:                                                       
	                                                                
	 GMST = GMST0 + UT                                              
	                                                                
	where GMST0 is the GMST "at 0h UT" but at the current moment!   
	Defined in this way, GMST0 will increase with about 4 min a     
	day.  It also happens that GMST0 (in degrees, 1 hr = 15 degr)   
	is equal to the Sun's mean longitude plus/minus 180 degrees!    
	(if we neglect aberration, which amounts to 20 seconds of arc   
	or 1.33 seconds of time)
	"""
	# Sidtime at 0h UT = L (Sun's mean longitude) + 180.0 degr  
	# L = M + w, as defined in sunpos().  Since I'm too lazy to 
	# add these numbers, I'll let the C compiler do it for me.  
	# Any decent C compiler will add the constants at compile   
	# time, imposing no runtime or code overhead.               
						
	sidtim0 = self.revolution((180.0 + 356.0470 + 282.9404) +
	                             (0.9856002585 + 4.70935E-5) * d)
	return sidtim0;
					 
	
	
if __name__ == "__main__":
    from time import *
    k = Sun()
#

    lat= -30.71317                # KAT longitude
    long= 21.4439                 # KAT latitude
    year=gmtime()[0]               #get current year
    yyyy=str(year)+" "
#   month=gmtime()[1]
#   day=gmtime()[2]

    for doy in range(1,365,7):            #loop over days of the relevant year per week
         date=strptime(yyyy+str(doy),"%Y %j")
         month=date[1]
         day=date[2]
#        print k.sunRiseSet(2002, 3, 22, 25.42, 62.15)
         rise,sunset=k.sunRiseSet(year, month, day, long, lat)
         daylight=sunset-rise
         print strftime("\nWeek%W Day%j %a %d %b", date),
         print "%4d/%02d/%02d: Sunrise %5.2f Set %5.2f UT"%(year,month,day,rise,sunset)
         d= k.daysSince2000Jan0(year,month,day)
    
         gmst=k.GMST0(d)             # gmst midnight on that day in DEG
         lst=(gmst+long)/15  # from degrees to hours
         print "daylight %.3f hrs" %(daylight),
         print ": At 0UT LST %5.2f" %(lst)
         lstris=(lst+rise*1.002737909350795)%24 # from UT hours to LST hours
         lstset=(lst+sunset*1.002737909350795)%24 # from UT hours to LST hours
         print("LST Sunrsie %5.2f  LST Sunset   %5.2f") %(lstris,lstset)

     
    


Week00 Day001 Thu 01 Jan 2015/01/01: Sunrise  3.58 Set 17.68 UT
daylight 14.097 hrs : At 0UT LST  8.12
LST Sunrsie 11.71  LST Sunset    1.84

Week01 Day008 Thu 08 Jan 2015/01/08: Sunrise  3.67 Set 17.69 UT
daylight 14.024 hrs : At 0UT LST  8.58
LST Sunrsie 12.26  LST Sunset    2.32

Week02 Day015 Thu 15 Jan 2015/01/15: Sunrise  3.77 Set 17.68 UT
daylight 13.917 hrs : At 0UT LST  9.04
LST Sunrsie 12.82  LST Sunset    2.77

Week03 Day022 Thu 22 Jan 2015/01/22: Sunrise  3.87 Set 17.65 UT
daylight 13.781 hrs : At 0UT LST  9.50
LST Sunrsie 13.38  LST Sunset    3.20

Week04 Day029 Thu 29 Jan 2015/01/29: Sunrise  3.98 Set 17.60 UT
daylight 13.622 hrs : At 0UT LST  9.96
LST Sunrsie 13.95  LST Sunset    3.61

Week05 Day036 Thu 05 Feb 2015/02/05: Sunrise  4.08 Set 17.52 UT
daylight 13.443 hrs : At 0UT LST 10.42
LST Sunrsie 14.51  LST Sunset    3.99

Week06 Day043 Thu 12 Feb 2015/02/12: Sunrise  4.18 Set 17.43 UT
daylight 13.250 hrs : At 0UT LST 10.88
LST Sunrsie 15.07  LST Sunset    4.36

Week0

In [ ]:
# 1 UT hour  =1.002 737 909 350 795 LST hours
LST_rise=(21.93+5.37*1.002737909350795)%24
LST_set=(21.93+15.99*1.002737909350795)%24
print ("LSTs sunrise %6.3f sunset %6.3f") %(LST_rise,LST_set)